In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV

/Users/danielfriar/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def accuracy(pred, actual):
    """Returns percentage of correctly classified labels"""
    return sum(pred==actual) / len(actual)

In [3]:
# Import data and split out labels
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Split into labels, names and data
y_train = train['class']
names_train = train['name']
X_train = train.drop(['class', 'name', 'sequence'], axis=1)

y_test = test['class']
names_train = test['name']
X_test = test.drop(['class', 'name', 'sequence'], axis=1)

# Further split train into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.20, random_state=0)

In [4]:
# Optimize a random forest model using grid search
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [20], 
    'max_depth': [10, 20, 30],
    'max_features': [9],
    'n_jobs': [9],
    'min_samples_leaf': [10, 20, 30]
}

grid_rf = GridSearchCV(rf, param_grid, cv=10, verbose=1)
grid_rf.fit(X_train, y_train)
print("#-------- DONE WITH GRID SEARCH.")
best_model = grid_rf.best_estimator_
best_params = grid_rf.best_params_ 
scores = grid_rf.grid_scores_
print(best_params)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
#-------- DONE WITH GRID SEARCH.

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:   45.0s finished
/Users/danielfriar/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)



{'max_features': 9, 'n_estimators': 20, 'n_jobs': 9, 'max_depth': 30, 'min_samples_leaf': 10}


In [6]:
# Calculate cross validation accuracy
rf = RandomForestClassifier(n_estimators=30, criterion='entropy', max_features=9, max_depth=30,
                            min_samples_leaf = 1, bootstrap=True, oob_score=True, n_jobs=9, random_state=0)
print(np.mean(cross_val_score(rf, X_train, y_train, cv=10)))

0.724839513001


In [47]:
# Fit to full training data and table feature importances
rf = rf.fit(X_train, y_train)
importances = rf.feature_importances_
importance = pd.DataFrame(importances, index=X_train.columns, columns=["importance"])
# importance.sort('importance', ascending=0)

In [49]:
# Print train and test accuracy
print("Training Accuracy = %f" % accuracy(rf.predict(X_train), y_train))
print("Test Accuracy = %f" % accuracy(rf.predict(X_test), y_test))

Training Accuracy = 0.844781
Test Accuracy = 0.723377
